In [1]:

from Point_Er import Point
from Lattice_Er import Lattice
from Simulator_Er import Simulator
from utils import *
from EnergyTransfer_Er import *
import numpy as np

tag_default={'c0':9.836062e-40, # Yb-Yb resonant energy transfer
        'Ws': 827,
        'E1E0':88.12753083306136+39,
        'E2E1':13.728308752002313+8.3+4077.51425913661,'E2E0':105.00663885847584,
        'E3E2':0.6904748414556272+1.7+353536.78966928925,'E3E1':40.06626483314129,'E3E0':107.07825106403719,
        'E4E3':1.4142534182563467+78491.28131241069,'E4E2':49.124834957391506,'E4E1':45.114305779338295,'E4E0':1009.6221517188111,
        'E5E4':0.5491077883920105+20115.437448364588,'E5E3':46.481404188403104,'E5E2':28.889483458690968,'E5E1':378.15231194559027,'E5E0':919.1044353717751,
        'E6E5':0.02617036285192619+73532852.08885323,'E6E4':8.841624545216064,'E6E3':47.60084543949401,'E6E2':41.09061870168263,'E6E1':71.35702052573745,'E6E0':2812.803587953125,
        'E7E6':0.4092613138830535+15062862.442801291,'E7E5':0.01904121955274265,'E7E4':3.4467583618029134+17.9,'E7E3':93.44157758618482,'E7E2':162.98778196545229,'E7E1':334.1120016219258,'E7E0':2256.2758284193}

# Progress file

In [2]:
import os
import pickle
from datetime import datetime
from time import sleep


# Function to load or initialize progress
def load_or_initialize_progress(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'rb') as f:
            return pickle.load(f)
    else:
        return {}

# Function to save progress
def save_progress(file_path, data):
    with open(file_path, 'wb') as f:
        pickle.dump(data, f)


# File path for progress tracking
progress_folder = 'Yuxuan_data_files'
if not os.path.exists(progress_folder):
    os.makedirs(progress_folder)

# Generate a unique filename
current_date = datetime.now().strftime('%m_%d_%Y')
base_filename = f'Progress_{current_date}'
index = 1
progress_filepath = os.path.join(progress_folder, f'{base_filename}_{index}.ipynb')
    
while os.path.exists(progress_filepath):
    index += 1
    progress_filepath = os.path.join(progress_folder, f'{base_filename}_{index}.ipynb')


# The first time to run the code, the folder 'Chuanyu_data_files' will have a new file: 'myC_progress.pkl'
# if there is no at least one combination finished, there would be no 'myC_progress.pkl' in the directory
# check the progress data if the program is interruptted


"""
file_path = 'Yuxuan_data_files/myC_progress.pkl'

with open(file_path, 'rb') as file:
    progress_data = pickle.load(file)

for key, value in progress_data.items():
    print(key, value)
"""

# Load or initialize progress data
# if progress_file_path doesn't exist, then return a {}
# if progress_file_path exists, then return a progress_data, which is an incomplete myC
# after interruption, run code and the the progress bar will start from the next combination

myC = load_or_initialize_progress(progress_filepath)


# Instruction file

In [3]:
import os
import nbformat as nbf
from datetime import datetime

def create_instruction_notebook(guide_folder='Yuxuan_data_files', saturation_plot_file='SaturationCurves.py', optimal_percentage_file='OptimalPercentage.py', population_evolution_file='PopulationEvolution.py'):
    """
    Create a Jupyter Notebook with instructions and code to load data and generate plots.
    
    Parameters:
    - guide_folder (str): Folder where the guide notebook will be saved.
    - saturation_plot_file (str): Path to the Python file containing the SaturationPlot class.
    - optimal_percentage_file (str): Path to the Python file containing the SinglePowerDensityPlot class.
    - population_evolution_file (str): Path to the Python file containing the PopulationEvolutionPlot class.
    """
    # Ensure the guide folder exists
    if not os.path.exists(guide_folder):
        os.makedirs(guide_folder)

    # Generate a unique filename for the notebook
    current_date = datetime.now().strftime('%m_%d_%Y')
    base_filename = f'Guide_{current_date}'
    index = 1
    notebook_path = os.path.join(guide_folder, f'{base_filename}_{index}.ipynb')
    while os.path.exists(notebook_path):
        index += 1
        notebook_path = os.path.join(guide_folder, f'{base_filename}_{index}.ipynb')

    # Ask for instructions to include in the notebook
    instructions = input("Please provide instructions for this dataset: ")

    # Create a new notebook object
    nb = nbf.v4.new_notebook()

    # Add a markdown cell with the provided instructions
    nb.cells.append(nbf.v4.new_markdown_cell(f"# Guide for Dataset - {current_date}_{index}\n\n{instructions}"))

    # Add a code cell for data loading
    nb.cells.append(nbf.v4.new_code_cell(f"""\
import pickle
import os


# Load the data

filepath = f'myC_{current_date}_{index}.pkl'
with open(filepath, 'rb') as f:
    data = pickle.load(f)

print("Data loaded successfully")
"""))

    # Add a code cell for importing and using the SaturationPlot class
    nb.cells.append(nbf.v4.new_code_cell(f"""\
# Import the SaturationPlot class from the SaturationCurves module
from {os.path.splitext(saturation_plot_file)[0]} import SaturationPlot

# Generate the saturation curves plot
plot = SaturationPlot(data)
plot.generate_plot(output_file='saturation_plot.html')
"""))

    # Add a code cell for importing and using the SinglePowerDensityPlot class
    nb.cells.append(nbf.v4.new_code_cell(f"""\
# Import the SinglePowerDensityPlot class from the OptimalPercentage module
from {os.path.splitext(optimal_percentage_file)[0]} import SinglePowerDensityPlot


# Provide a list of available power densities
available_power_densities = sorted({{k for subdict in data.values() for k in subdict.keys()}})
print(f'Available power densities: {{available_power_densities}}')

# Prompt the user to select a power density
selected_power_density = float(input(f'Input a power density from the above options: '))

# Generate the single power density plot
single_plot = SinglePowerDensityPlot(data, selected_power_density)
single_plot.generate_plot(output_file='single_power_density_plot.html')
"""))

    # Add a code cell for importing and using the PopulationEvolutionPlot class
    nb.cells.append(nbf.v4.new_code_cell(f"""\
# Import the PopulationEvolutionPlot class from the PopulationEvolution module
from {os.path.splitext(population_evolution_file)[0]} import PopulationEvolutionPlot

# Provide a list of available percentages

available_percentages = sorted(data.keys())
print(f'Available percentages: {{available_percentages}}')
# Prompt the user to select a percentage
percentage = float(input(f'Input a percentage from the above options: '))

available_power_densities = sorted({{k for k in data[percentage].keys()}})
print(f'Available power densities for percentage {{percentage}}: {{available_power_densities}}')
# Prompt the user to select a power density
power_density = float(input(f'Input a power density from the above options: '))




# Generate the population evolution plot
pop_plot = PopulationEvolutionPlot(data, percentage, power_density)
pop_plot.generate_plot(output_file='population_evolution_plot.html')
"""))

    # Write the notebook to the file
    with open(notebook_path, 'w') as f:
        nbf.write(nb, f)

    print(f"Instruction notebook has been successfully saved to '{notebook_path}'")
    print()


    


In [4]:
create_instruction_notebook('Yuxuan_data_files', 'SaturationCurves.py', 'OptimalPercentage.py', 'PopulationEvolution.py')

Instruction notebook has been successfully saved to 'Yuxuan_data_files\Guide_05_12_2024_1.ipynb'



# Save file

In [5]:
# use pickle module to save the results as a pickle file and reload them later
# The / character cannot be used in file names because it is reserved as a directory separator, replace / with underscores _ or using another safe delimiter.


import pickle
from datetime import datetime
import os
import plotly.graph_objects as go
import nbformat as nbf



class PickleSaver:

    def __init__(self, base_name='Mydata', folder='Yuxuan_data_files'):
     
        self.base_name = base_name
        self.folder = folder

        # Ensure the folder exists
        if not os.path.exists(self.folder):
            os.makedirs(self.folder)

    def save_data(self, data):
      
        # Create a base filename using the current date
        current_date = datetime.now().strftime('%m_%d_%Y')
        base_filename = f'{self.base_name}_{current_date}'
        
        index = 1
        filename = os.path.join(self.folder, f'{base_filename}_{index}.pkl')
        while os.path.exists(filename):
            index += 1
            filename = os.path.join(self.folder, f'{base_filename}_{index}.pkl')


        # Save data to a pickle file with the generated filename
        with open(filename, 'wb') as f:
            pickle.dump(data, f)

        print(f"Data has been successfully saved to '{filename}'")
        print()


    def load_all_data(self):
        
        if not os.path.exists(self.folder):
            print(f"Folder '{self.folder}' does not exist.")
            return {}

        files = [f for f in os.listdir(self.folder) if f.endswith('.pkl')]
        all_data = {}

        for file in sorted(files):
            filepath = os.path.join(self.folder, file)
            with open(filepath, 'rb') as f:
                data = pickle.load(f)
                all_data[file] = data

        print(f'Filename = {file}')

        KEY1=[]
        KEY2=[]

        for key1, value1 in data.items():
            KEY1.append(key1)
        print(f'\nAll percentages = {KEY1}')

        for key1, value1 in data.items():
            for key2, value2 in value1.items():
                KEY2.append(key2)
            break
        print(f'\nFor each percentage, all power densities = {KEY2}')

        # Use a set to keep track of unique third keys
        unique_third_keys = set()

        print('\nFor each combination of Percentage+Power density, the accessible data: ')
        print()
        for key1, value1 in data.items():
            for key2, value2 in value1.items():
                for key3 in value2.keys():
                    if key3 not in unique_third_keys:
                        unique_third_keys.add(key3)
                        print(f'key3 = {key3}, value3 = {value2[key3]}')

# Execution

In [7]:



# nanoparticle diameter: 8 nm, critical distance: 1 nm
d=8
shell=0
r0=1


# Er_conc=[0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6]
# Er_conc=[0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8]
# Er_conc=[0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]
# Er_conc=[0.04, 0.06, 0.08, 0.1, 0.12, 0.15, 0.2, 0.5]
# Yb_conc=0.2

Er_conc = [0.10] # [0.04, 0.06, 0.08, 0.1, 0.12, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.5, 0.6, 0.7, 0.8]
power_density = [3*10**4] # [1*10**3, 2*10**3, 4*10**3, 8*10**3, 2*10**4, 4*10**4, 1*10**5, 1*10**6]

# emitter_conc = [0.04, 0.06, 0.08, 0.1, 0.12, 0.15, 0.2, 0.5]
# power_density = [1.2*10**3, 1.8*10**3, 2.8*10**3, 4.4*10**3, 5*10**3, 7.5*10**3, 1.1*10**4, 1.4*10**4, 1.7*10**4, 2*10**4, 2.3*10**4, 2.7*10**4, 3*10**4]


t1 = 2000
t2 = 4000


# Simulate and track progress
for c in Er_conc:
    if c not in myC:
        myC[c] = {}

    for p in power_density:
        if p not in myC[c]:
            tag_default['laser_er'] = tag_default['laser'] / 10
            print(f"Running simulation for Er concentration {c}, power density {p}")

            # Mocking tag update and simulation
            tag_default['laser'] = 0.0615 * p    
            
            my_lattice = Lattice(0.2, c, d, r0)  # shell
            my_simulator = Simulator(my_lattice, tag=tag_default)
            result = my_simulator.simulate(t1, t2)  # r is a dictionary from 'sim_stats'

            # Store result and save progress
            myC[c][p] = result
            # store the current progress to the filepath
            save_progress(progress_filepath, myC)

            # Simulate time delay to represent real simulation time
            sleep(1)

print('\nAll progress have been finished.')

Running simulation for Er concentration 0.1, power density 30000


100%|██████████| 2000/2000 [00:17<00:00, 116.82it/s]



All progress have been finished.


In [8]:
# after all the progress are finished
# The first time to run the code, the folder 'Chuanyu_data_files' will have a new file: 'myC_05_06_2024_1.pkl', which store all data
# The second time to run the code, the folder 'Chuanyu_data_files' will have a new file: 'myC_05_06_2024_2.pkl', which store all data

saver = PickleSaver(base_name='myC', folder='Yuxuan_data_files')
saver.save_data(myC)

# then, call the method with myC, to see the data structure
data_structure = saver.load_all_data()

Data has been successfully saved to 'Yuxuan_data_files\myC_05_12_2024_1.pkl'

Filename = myC_05_12_2024_1.pkl

All percentages = [0.1]

For each percentage, all power densities = [30000]

For each combination of Percentage+Power density, the accessible data: 

key3 = red_microsecond, value3 = [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

: 

---
### If you want to run the code again, firstly make sure you have run the code above to store the data:

'''
saver = PickleSaver(base_name='myC', folder='Chuanyu_data_files')
saver.save_data(myC)b
all_save_data = saver.load_all_data()
'''
#### In the same directory, there would be a new file: myC_05_08_2024_1, which save all data from this simulation
#### Then you can detele the progress file and to run the code
---